In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, LeakyReLU
from keras.utils import np_utils
from keras import optimizers
from keras import regularizers
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

/kaggle/input/audio-binary-classification/train_data.npy
/kaggle/input/audio-binary-classification/train_labels.csv
/kaggle/input/audio-binary-classification/sample_submission.csv
/kaggle/input/audio-binary-classification/test_data.npy


/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [2]:
train_data = np.load('/kaggle/input/audio-binary-classification/train_data.npy');
test_data = np.load('/kaggle/input/audio-binary-classification/test_data.npy');


print("inputs loaded")

inputs loaded


In [3]:
#Scale and Reshape
train_data = train_data.astype('float32');
test_data = test_data.astype('float32');

#Normalize Data
train_data /= np.amax(test_data);
test_data /= np.amax(test_data);

print(train_data.shape);

train_data = train_data.reshape(-1,44100,1);
test_data = test_data.reshape(-1,44100,1);

input_shape = train_data.shape;
print(input_shape);

(3999, 44100)
(3999, 44100, 1)


In [4]:
#Look at CSV and store as array
train_labels = pd.read_csv("/kaggle/input/audio-binary-classification/train_labels.csv");
train_labels = train_labels.values;
train_labels = train_labels[:,1];
print(train_labels[:10]);
print(train_labels.shape)

[0 0 0 1 0 0 1 0 1 0]
(3999,)


In [5]:
# create convolution neural network
model = Sequential()
model.add(Conv1D(32, kernel_size=(10),
                 input_shape=(44100,1)))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.25))
model.add(Conv1D(64, (8)))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling1D(pool_size=(4)))
model.add(Dropout(0.25))
model.add(Conv1D(64, (2)))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling1D(pool_size=(4)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary();

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 44091, 32)         352       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 44091, 32)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 11022, 32)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 11022, 32)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 11015, 64)         16448     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 11015, 64)         0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 2753, 64)          0         
__________

In [6]:
#Compile model
sgd = optimizers.SGD(lr=.01);
model.compile(loss='MSE', 
              optimizer=sgd,
              metrics=['accuracy'])

In [7]:
history = model.fit(train_data,train_labels,
          batch_size=25,epochs=100,verbose=1)

Epoch 1/100
3999/3999 [==============================] - 7s 2ms/step - loss: 0.1406 - acc: 0.8615
Epoch 2/100
3999/3999 [==============================] - 4s 1ms/step - loss: 0.1110 - acc: 0.8780
Epoch 3/100
3999/3999 [==============================] - 4s 1ms/step - loss: 0.1100 - acc: 0.8780
Epoch 4/100
3999/3999 [==============================] - 4s 1ms/step - loss: 0.1089 - acc: 0.8780
Epoch 5/100
3999/3999 [==============================] - 4s 1ms/step - loss: 0.1088 - acc: 0.8780
Epoch 6/100
3999/3999 [==============================] - 4s 1ms/step - loss: 0.1098 - acc: 0.8780
Epoch 7/100
3999/3999 [==============================] - 4s 1ms/step - loss: 0.1087 - acc: 0.8780
Epoch 8/100
3999/3999 [==============================] - 4s 1ms/step - loss: 0.1077 - acc: 0.8780
Epoch 9/100
3999/3999 [==============================] - 4s 1ms/step - loss: 0.1071 - acc: 0.8780
Epoch 10/100
3999/3999 [==============================] - 4s 1ms/step - loss: 0.1072 - acc: 0.8780
Epoch 11/100
3999/3

In [8]:
Y_pred = model.predict(test_data);
print(Y_pred)
df_pred = pd.DataFrame(Y_pred);
df_pred.index.name = "Id"
df_pred = pd.DataFrame.rename(df_pred,columns={0:"Label"})
print(df_pred);
df_pred.to_csv("output.csv")

[[0.01163042]
 [0.04114884]
 [0.05694044]
 ...
 [0.01088113]
 [0.00194567]
 [0.4783548 ]]
         Label
Id            
0     0.011630
1     0.041149
2     0.056940
3     0.006271
4     0.007454
...        ...
3992  0.063679
3993  0.005362
3994  0.010881
3995  0.001946
3996  0.478355

[3997 rows x 1 columns]
